In [ ]:
import numpy as np
import os
import glob
from DET import DET

scores_path = 'similarity_scores/'

plot_root = 'plots_DET/'

if not os.path.exists(plot_root):
    os.mkdir(plot_root)

In [ ]:
filtered_files = []
order = ['real','real_vs_synthetic_cap','real_vs_synthetic_glass','real_vs_synthetic_sunglass', 'real_vs_synthetic_mask']

for file in [os.path.basename(file) for file in glob.glob(scores_path + '*.npy')]:
    filename = os.path.splitext(file)[0]
    #print(filename)
    if 'real' and not 'synthetic' in filename:
        filtered_files.append('_'.join(filename.split('_')[:-1]))
    elif len(file.split('_')[3:-1]) == 1:
        filtered_files.append('_'.join(filename.split('_')[:-1]))
filtered_files = [*set(filtered_files)]
filtered_files = sorted(filtered_files, key=order.index)

det = DET(biometric_evaluation_type='algorithm')
det.create_figure()
for file in filtered_files:
    non_mated = np.load(os.path.join(scores_path,file+'_non-mated.npy'))
    mated = np.load(os.path.join(scores_path,file+'_mated.npy'))
    if len(file.split('_')[3:]) == 1:
        system = file.split('_')[3:][0]
    else:
        system = 'baseline'
    det.plot(tar=mated, non=non_mated, label=system)
det.legend_on()
det.save(plot_root + 'baseline_single_occ', 'pdf')

In [ ]:
filters = ['_mask_m','_mask_sunglass_m','_mask_sunglass_cap_m']
order = ['baseline','mask','mask_sunglass','mask_sunglass_cap']
filtered_files = []

for file in [os.path.basename(file) for file in glob.glob(scores_path + '*_mated.npy')]:
    filename = os.path.splitext(file)[0]
    if 'real' and not 'synthetic' in filename:
        filtered_files.append(['baseline','_'.join(filename.split('_')[:-1])])
    elif any(substring in filename for substring in filters):
        filtered_files.append(['_'.join(filename.split('_')[3:-1]), '_'.join(filename.split('_')[:-1])])
filtered_files = sorted(filtered_files, key=lambda x: x[1])
det = DET(biometric_evaluation_type='algorithm')
det.x_limits = np.array([1e-7, .95])
det.y_limits = np.array([1e-7, .95])
det.create_figure()
for legend, file in filtered_files:
    non_mated = np.load(os.path.join(scores_path,file+'_non-mated.npy'))
    mated = np.load(os.path.join(scores_path,file+'_mated.npy'))
    legend = ', '.join(legend.split('_'))
    det.plot(tar=mated, non=non_mated, label=legend)
det.legend_on()
det.save(plot_root + 'baseline_selected_occ', 'pdf')

In [ ]:
det = DET()
det.x_limits = np.array([1e-7, .95])
det.y_limits = np.array([1e-7, .95])
det.create_figure()
if os.path.exists(scores_path+'real_mated.npy') and os.path.exists(scores_path+'real_non-mated.npy'):
    mated = np.load(os.path.join(scores_path,'real_mated.npy'))
    non_mated = np.load(os.path.join(scores_path,'real_non-mated.npy'))
    det.plot(tar=mated, non=non_mated, label='original dataset')

if os.path.exists(scores_path+'real_vs_synthetic_mated.npy') and os.path.exists(scores_path+'real_vs_synthetic_non-mated.npy'):
    mated = np.load(os.path.join(scores_path,'real_vs_synthetic_mated.npy'))
    non_mated = np.load(os.path.join(scores_path,'real_vs_synthetic_non-mated.npy'))
    det.plot(tar=mated, non=non_mated, label='occluded dataset')

det.legend_on()
det.save(plot_root + 'dataset-occluded_dataset', 'pdf')